In [2]:
import os
from os.path import join, split, isfile

image_dir = 'D:\\aic19\\aic19-track3-test-data\\aic19-track3-segmentation\\aic19-track3-frames'

only_images = [f for f in os.listdir(image_dir) if isfile(join(image_dir, f)) and f[-4:] == '.jpg']
only_images.sort()
only_images

['001.jpg',
 '002.jpg',
 '003.jpg',
 '004.jpg',
 '005.jpg',
 '006.jpg',
 '007.jpg',
 '008.jpg',
 '009.jpg',
 '010.jpg',
 '011.jpg',
 '012.jpg',
 '013.jpg',
 '014.jpg',
 '015.jpg',
 '016.jpg',
 '017.jpg',
 '018.jpg',
 '019.jpg',
 '020.jpg',
 '021.jpg',
 '022.jpg',
 '023.jpg',
 '024.jpg',
 '025.jpg',
 '026.jpg',
 '027.jpg',
 '028.jpg',
 '029.jpg',
 '030.jpg',
 '031.jpg',
 '032.jpg',
 '033.jpg',
 '034.jpg',
 '035.jpg',
 '036.jpg',
 '037.jpg',
 '038.jpg',
 '039.jpg',
 '040.jpg',
 '041.jpg',
 '042.jpg',
 '043.jpg',
 '044.jpg',
 '045.jpg',
 '046.jpg',
 '047.jpg',
 '048.jpg',
 '049.jpg',
 '050.jpg',
 '051.jpg',
 '052.jpg',
 '053.jpg',
 '054.jpg',
 '055.jpg',
 '056.jpg',
 '057.jpg',
 '058.jpg',
 '059.jpg',
 '060.jpg',
 '061.jpg',
 '062.jpg',
 '063.jpg',
 '064.jpg',
 '065.jpg',
 '066.jpg',
 '067.jpg',
 '068.jpg',
 '069.jpg',
 '070.jpg',
 '071.jpg',
 '072.jpg',
 '073.jpg',
 '074.jpg',
 '075.jpg',
 '076.jpg',
 '077.jpg',
 '078.jpg',
 '079.jpg',
 '080.jpg',
 '081.jpg',
 '082.jpg',
 '083.jpg',
 '08

In [ ]:
# k-means only

import cv2
import numpy as np

output_dir = 'D:\\aic19\\aic19-track3-test-data\\aic19-track3-segmentation\\aic19-track3-frames-kmeans'
os.makedirs(output_dir)

for f in only_images:
    print(f)
    img = cv2.imread(join(image_dir, f))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    Z = hsv.reshape((-1,3))
    # convert to np.float32
    Z = np.float32(Z)
    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 4
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((img.shape))
    img_new = cv2.cvtColor(res2, cv2.COLOR_HSV2BGR)
    cv2.imwrite(join(output_dir, f[:-4]+'-kmeans.jpg'), img_new)
    
    # define range of grass in HSV # H in opencv is in (0, 180)
    lower = np.array([25,36,32])
    upper = np.array([90,255,255])

    # Threshold the HSV image to get grass
    mask = cv2.inRange(res2, lower, upper)
    
    cv2.imwrite(join(output_dir, f[:-4]+'-grass-mask.jpg'), mask)
    
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    img_new = img/2 + mask/2
    cv2.imwrite(join(output_dir, f[:-4]+'-grass-img.jpg'), img_new)
    
    cv2.imshow('img_new', img_new)
    cv2.waitKey(1000)
cv2.destroyAllWindows()

In [102]:
# train SVM

import cv2
import numpy as np

labels = []
trainData = []

# road
img_road = cv2.imread('D:\\aic19\\aic19-track3-test-data\\aic19-track3-segmentation\\road.png')
img_road = cv2.cvtColor(img_road, cv2.COLOR_BGR2HSV)
h, w, c = img_road.shape
count_road = 0
for i in range(img_road.shape[0]):
    for j in range(img_road.shape[1]):
        #if sum(img_road[i,j]) < 255*3:
        if img_road[i,j,2] < 255:
            labels.append(-1)
            trainData.append(img_road[i,j])
            count_road += 1

# grass
img_grass = cv2.imread('D:\\aic19\\aic19-track3-test-data\\aic19-track3-segmentation\\grass.png')
img_grass = cv2.cvtColor(img_grass, cv2.COLOR_BGR2HSV)
h, w, c = img_grass.shape
count_grass = 0
for i in range(img_grass.shape[0]):
    for j in range(img_grass.shape[1]):
        #if sum(img_grass[i,j]) < 255*3:
        if img_grass[i,j,2] < 255:
            labels.append(1)
            trainData.append(img_grass[i,j])
            count_grass += 1

print('count grass: %d' % count_grass)
print('count road: %d' % count_road)
labels= np.array(labels, dtype=np.int32)
trainData = np.array(trainData, dtype=np.float32)
print('labels.shape = ' + str(labels.shape))
print('trainData.shape = ' + str(trainData.shape))
            
# train SVM
svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
#svm.setKernel(cv2.ml.SVM_RBF)
svm.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6))
svm.train(trainData, cv2.ml.ROW_SAMPLE, labels)

count grass: 79751
count road: 274073
labels.shape = (353824,)
trainData.shape = (353824, 3)


True

In [109]:
# predict by SVM (kmeans)

import os

output_dir = 'D:\\aic19\\aic19-track3-test-data\\aic19-track3-segmentation\\aic19-track3-frames-svm-k4'
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

for f in only_images:
    print(f)
    img = cv2.imread(join(image_dir, f))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    Z = hsv.reshape((-1,3))
    # convert to np.float32
    Z = np.float32(Z)
    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 4
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((img.shape))
    
    # classify using SVM
    sample = res2.reshape(-1, 3).astype(np.float32)
    
    _, response = svm.predict(sample)
    response = response.reshape(img.shape[0], img.shape[1])
    
    indices_grass = np.argwhere(response < 0)
    indices_road = np.argwhere(response > 0)
    
    mask_grass = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    mask_grass[indices_grass[:,0] ,indices_grass[:,1]] = 255
    
    mask_road = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    mask_road[indices_road[:,0] ,indices_road[:,1]] = 255
    
    
    res2 = cv2.cvtColor(res2, cv2.COLOR_HSV2BGR)
    cv2.imwrite(join(output_dir, f[:-4]+'-kmeans.jpg'), res2)
    cv2.imwrite(join(output_dir, f[:-4]+'-grass-mask.png'), mask_grass)
    cv2.imwrite(join(output_dir, f[:-4]+'-road-mask.png'), mask_road)
    
    mask_grass = cv2.cvtColor(mask_grass, cv2.COLOR_GRAY2BGR)
    img_grass = img/2 + mask_grass/2
    cv2.imwrite(join(output_dir, f[:-4]+'-grass-img.jpg'), img_grass)

    mask_road = cv2.cvtColor(mask_road, cv2.COLOR_GRAY2BGR)
    img_road = img/2 + mask_road/2
    cv2.imwrite(join(output_dir, f[:-4]+'-road-img.jpg'), img_road)

    #cv2.imshow('img_grass', img_grass)
    #cv2.waitKey(1000)
#cv2.destroyAllWindows()

001.jpg
002.jpg
003.jpg
004.jpg
005.jpg
006.jpg
007.jpg
008.jpg
009.jpg
010.jpg
011.jpg
012.jpg
013.jpg
014.jpg
015.jpg
016.jpg
017.jpg
018.jpg
019.jpg
020.jpg
021.jpg
022.jpg
023.jpg
024.jpg
025.jpg
026.jpg
027.jpg
028.jpg
029.jpg
030.jpg
031.jpg
032.jpg
033.jpg
034.jpg
035.jpg
036.jpg
037.jpg
038.jpg
039.jpg
040.jpg
041.jpg
042.jpg
043.jpg
044.jpg
045.jpg
046.jpg
047.jpg
048.jpg
049.jpg
050.jpg
051.jpg
052.jpg
053.jpg
054.jpg
055.jpg
056.jpg
057.jpg
058.jpg
059.jpg
060.jpg
061.jpg
062.jpg
063.jpg
064.jpg
065.jpg
066.jpg
067.jpg
068.jpg
069.jpg
070.jpg
071.jpg
072.jpg
073.jpg
074.jpg
075.jpg
076.jpg
077.jpg
078.jpg
079.jpg
080.jpg
081.jpg
082.jpg
083.jpg
084.jpg
085.jpg
086.jpg
087.jpg
088.jpg
089.jpg
090.jpg
091.jpg
092.jpg
093.jpg
094.jpg
095.jpg
096.jpg
097.jpg
098.jpg
099.jpg
100.jpg


In [107]:
# predict by SVM (every pixel)

import os

output_dir = 'D:\\aic19\\aic19-track3-test-data\\aic19-track3-segmentation\\aic19-track3-frames-svm'
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

for f in only_images:
    print(f)
    img = cv2.imread(join(image_dir, f))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # classify using SVM
    sample = hsv.reshape(-1, 3).astype(np.float32)
    
    _, response = svm.predict(sample)
    response = response.reshape(img.shape[0], img.shape[1])
    
    indices_grass = np.argwhere(response < 0)
    indices_road = np.argwhere(response > 0)
    
    mask_grass = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    mask_grass[indices_grass[:,0] ,indices_grass[:,1]] = 255
    
    mask_road = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    mask_road[indices_road[:,0] ,indices_road[:,1]] = 255
    
    cv2.imwrite(join(output_dir, f[:-4]+'.jpg'), img)
    cv2.imwrite(join(output_dir, f[:-4]+'-grass-mask.png'), mask_grass)
    cv2.imwrite(join(output_dir, f[:-4]+'-road-mask.png'), mask_road)
    
    mask_grass = cv2.cvtColor(mask_grass, cv2.COLOR_GRAY2BGR)
    img_grass = img/2 + mask_grass/2
    cv2.imwrite(join(output_dir, f[:-4]+'-grass-img.jpg'), img_grass)

    mask_road = cv2.cvtColor(mask_road, cv2.COLOR_GRAY2BGR)
    img_road = img/2 + mask_road/2
    cv2.imwrite(join(output_dir, f[:-4]+'-road-img.jpg'), img_road)

001.jpg
002.jpg
003.jpg
004.jpg
005.jpg
006.jpg
007.jpg
008.jpg
009.jpg
010.jpg
011.jpg
012.jpg
013.jpg
014.jpg
015.jpg
016.jpg
017.jpg
018.jpg
019.jpg
020.jpg
021.jpg
022.jpg
023.jpg
024.jpg
025.jpg
026.jpg
027.jpg
028.jpg
029.jpg
030.jpg
031.jpg
032.jpg
033.jpg
034.jpg
035.jpg
036.jpg
037.jpg
038.jpg
039.jpg
040.jpg
041.jpg
042.jpg
043.jpg
044.jpg
045.jpg
046.jpg
047.jpg
048.jpg
049.jpg
050.jpg
051.jpg
052.jpg
053.jpg
054.jpg
055.jpg
056.jpg
057.jpg
058.jpg
059.jpg
060.jpg
061.jpg
062.jpg
063.jpg
064.jpg
065.jpg
066.jpg
067.jpg
068.jpg
069.jpg
070.jpg
071.jpg
072.jpg
073.jpg
074.jpg
075.jpg
076.jpg
077.jpg
078.jpg
079.jpg
080.jpg
081.jpg
082.jpg
083.jpg
084.jpg
085.jpg
086.jpg
087.jpg
088.jpg
089.jpg
090.jpg
091.jpg
092.jpg
093.jpg
094.jpg
095.jpg
096.jpg
097.jpg
098.jpg
099.jpg
100.jpg
